In [2]:
import pandas as pd
df = pd.read_csv('C:/Users/chira/Desktop/Fashion Visual Search & Intelligent Styling Assistant/dataset/links.csv')  # Replace with your actual file

In [3]:
df

,pdp_images_s3,category_type
0,['https://gallery.stylumia.com/originals/2020/...,dress
1,['https://gallery.stylumia.com/originals/2020/...,dress
2,['https://gallery.stylumia.com/originals/2020/...,dress
3,['https://gallery.stylumia.com/originals/2020/...,dress
4,['https://gallery.stylumia.com/originals/2021/...,dress
...,...,...
17478,['https://gallery.stylumia.com/originals/2025/...,jeans
17479,['https://gallery.stylumia.com/originals/2025/...,jeans
17480,['https://gallery.stylumia.com/originals/2025/...,jeans
17481,['https://gallery.stylumia.com/originals/2025/...,jeans


# Embedding code below

## We have some missing values example - 
(1%|          | 152/17483 [01:41<2:24:44,  2.00it/s]
❌ Error for https://gallery.stylumia.com/originals/2022/12/383/51d412a50238008396b4d8a047d24f2a0a9343447486c48bdfa50e1f59c9545d_1.jpg: HTTPSConnectionPool(host='gallery.stylumia.com', port=443): Max retries exceeded with url: /originals/2022/12/383/51d412a50238008396b4d8a047d24f2a0a9343447486c48bdfa50e1f59c9545d_1.jpg (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_BAD_RECORD_MAC] sslv3 alert bad record mac (_ssl.c:2546)')))) Due to some network issue while GET requeest i assume its not more than 40+ imgs embeddings, so its shouldnt be a problem

In [3]:
import torch
import pandas as pd
import numpy as np
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from transformers import CLIPProcessor, CLIPModel
import requests
from io import BytesIO
import os
import ast  # for converting string list to real list

# ✅ Load CLIP model and processor
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:
df = pd.read_csv("C:/Users/chira/Desktop/Fashion Visual Search & Intelligent Styling Assistant/scripts/links.csv")

# Convert string list to actual list and extract first URL
def extract_first_url(url_list_str):
    try:
        url_list = ast.literal_eval(url_list_str)
        if isinstance(url_list, list) and len(url_list) > 0:
            return url_list[0]
    except:
        return None
    return None

df['image_url'] = df['pdp_images_s3'].apply(extract_first_url)

# Image embedding function
def get_image_embedding(url):
    try:
        response = requests.get(url, timeout=10)
        img = Image.open(BytesIO(response.content)).convert("RGB")
        inputs = processor(images=img, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model.get_image_features(**inputs)
        embedding = outputs[0].cpu().numpy()
        return embedding
    except Exception as e:
        print(f"❌ Error for {url}: {e}")
        return None

# Generate embeddings
embeddings = []
valid_rows = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    url = row['image_url']
    if not isinstance(url, str) or not url.startswith("http"):
        continue

    emb = get_image_embedding(url)
    if emb is not None:
        embeddings.append(emb)
        valid_rows.append(row)

# Set save directory
save_dir = "C:/Users/chira/Desktop/Fashion Visual Search & Intelligent Styling Assistant/embedding"
os.makedirs(save_dir, exist_ok=True)

# Save embeddings and valid data
if embeddings:
    embeddings_array = np.vstack(embeddings)
    np.save(os.path.join(save_dir, "fashion_image_embeddings.npy"), embeddings_array)
    pd.DataFrame(valid_rows).reset_index(drop=True).to_csv(os.path.join(save_dir, "valid_products.csv"), index=False)
    print(f"✅ Saved embeddings to: {os.path.join(save_dir, 'fashion_image_embeddings.npy')}")
    print(f"✅ Saved valid product data to: {os.path.join(save_dir, 'valid_products.csv')}")
else:
    print("⚠️ No embeddings were generated. Check URLs again.")


  1%|          | 152/17483 [01:41<2:24:44,  2.00it/s]

❌ Error for https://gallery.stylumia.com/originals/2022/12/383/51d412a50238008396b4d8a047d24f2a0a9343447486c48bdfa50e1f59c9545d_1.jpg: HTTPSConnectionPool(host='gallery.stylumia.com', port=443): Max retries exceeded with url: /originals/2022/12/383/51d412a50238008396b4d8a047d24f2a0a9343447486c48bdfa50e1f59c9545d_1.jpg (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_BAD_RECORD_MAC] sslv3 alert bad record mac (_ssl.c:2546)')))


  2%|▏         | 307/17483 [03:27<2:54:37,  1.64it/s]

❌ Error for https://gallery.stylumia.com/originals/2023/05/300/7015b588d4163fb7721bd58578613c4a389ec30ca6cc7c658debc7a2b28e2d0a_1.jpg: HTTPSConnectionPool(host='gallery.stylumia.com', port=443): Max retries exceeded with url: /originals/2023/05/300/7015b588d4163fb7721bd58578613c4a389ec30ca6cc7c658debc7a2b28e2d0a_1.jpg (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_BAD_RECORD_MAC] sslv3 alert bad record mac (_ssl.c:2546)')))


  2%|▏         | 367/17483 [04:10<2:35:49,  1.83it/s]

...

❌ Error for https://gallery.stylumia.com/originals/2025/03/12/f8476f5654a9be19e4c340b91332c60b3a9a6e1a77edd933da4766d954665856_1.jpg: HTTPSConnectionPool(host='gallery.stylumia.com', port=443): Max retries exceeded with url: /originals/2025/03/12/f8476f5654a9be19e4c340b91332c60b3a9a6e1a77edd933da4766d954665856_1.jpg (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_BAD_RECORD_MAC] sslv3 alert bad record mac (_ssl.c:2546)')))


 99%|█████████▉| 17384/17483 [3:22:32<00:46,  2.13it/s]

❌ Error for https://gallery.stylumia.com/originals/2025/04/299/34dbffcfbfd611d0552ef0549d93a828a4b21d724656d5303c623696fe45138f_1.jpg: HTTPSConnectionPool(host='gallery.stylumia.com', port=443): Max retries exceeded with url: /originals/2025/04/299/34dbffcfbfd611d0552ef0549d93a828a4b21d724656d5303c623696fe45138f_1.jpg (Caused by SSLError(SSLError(1, '[SSL: SSLV3_ALERT_BAD_RECORD_MAC] sslv3 alert bad record mac (_ssl.c:2546)')))


100%|██████████| 17483/17483 [3:23:38<00:00,  1.43it/s]


✅ Saved embeddings to: C:/Users/chira/Desktop/Fashion Visual Search & Intelligent Styling Assistant/embedding\fashion_image_embeddings.npy
✅ Saved valid product data to: C:/Users/chira/Desktop/Fashion Visual Search & Intelligent Styling Assistant/embedding\valid_products.csv
